# 1. TF-IDF + Random Forest

# 對比四種方法| **方法**                  | **特徵工程**                | **模型**                    | **準確率預估** | **優勢**                                     | **劣勢**                                      | **GPU 支援**            |
|--------------------------|----------------------------|-----------------------------|----------------|---------------------------------------------|----------------------------------------------|-------------------------|
| **TF-IDF + 隨機森林**      | 稀疏特徵表示，詞頻與逆文檔頻率權重 | 隨機森林                    | 75%-82%       | 模型穩定性強，對噪聲和高維數據不敏感          | 無法處理非線性模式，對語義信息利用不足           | 不支持                  |
| **TF-IDF + Boosting**      | 稀疏特徵表示，詞頻與逆文檔頻率權重 | XGBoost 或 LightGBM         | 78%-85%       | 擅長處理稀疏特徵，對錯分樣本有良好適應能力      | 訓練成本略高，需調參以達到最佳效果              | 支持（顯著加速，適合大數據集）|
| **Word2Vec + 隨機森林**     | 詞嵌入，計算句向量平均值       | 隨機森林                    | 72%-80%       | 能結合詞嵌入語義特徵，提升語義捕捉能力          | 詞嵌入需預處理，隨機森林對非線性語義的處理有限     | 不支持                  |
| **Word2Vec + CNN**         | 詞嵌入，保留語序            | 卷積神經網絡                | 75%-85%       | 捕捉局部語義特徵，對短文本效果佳               | 訓練需較多資源，對長文本效果有限               | 支持（顯著加速）         |
| **BERT 嵌入 + Transformer**| 上下文語義嵌入，保留全局語義 | 預訓練 BERT 模型             | 85%-90%       | 能捕捉上下文語義，分類準確率最高               | 訓練和推理成本高，需要大量數據和資源支持         | 支持（必要，否則速慢） |     |


In [ ]:
#kaggle 前置作業
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# 引入必要的庫
import json
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# 這部分請小心執行，這是 "未做過"  雜訊處理的程式碼

In [ ]:
# ======== 資料讀取與整理 ========
# 1. 讀取 JSON 格式的推文數據
data = []
with open('/kaggle/input/dm-2024-isa-5810-lab-2-homework/tweets_DM.json', 'r') as f:
    for line in f:
        data.append(json.loads(line))
f.close()

# 這部分請小心執行，這是 "做過" 雜訊處理的程式碼

In [ ]:
import json
import re
import emoji
import pandas as pd

# 定義表情符號到關鍵字的映射字典
emoji_dict = {
    '😂': '[joy]',
    '❤️': '[love]',
    '😍': '[adoration]',
    '😭': '[cry]',
    '❤': '[care]',
    '😊': '[happy]',
    '🙏': '[pray]',
    '😘': '[kiss]',
    '💕': '[love_each_other]',
    '🔥': '[fire]',
    '😩': '[weary]',
    '🤔': '[think]',
    '💯': '[perfect]',
    '💙': '[loyalty]',
    '🙄': '[annoyed]',
    '😁': '[happy]',
    '🙌': '[celebrate]',
    '🙏🏾': '[pray]',
    '👍': '[approve]',
    '🙏🏽': '[pray]'
}

# 定義清理推文文本的函數
def clean_tweet(text, emoji_dict):
    # 將定義的表情符號替換為對應的關鍵詞
    for emj, keyword in emoji_dict.items():
        text = text.replace(emj, keyword)
    # 移除其餘的表情符號
    text = emoji.replace_emoji(text, replace='')
    # 移除 <LH> 標籤
    text = re.sub(r'<LH>', '', text)
    # 移除多餘的空白字元
    text = text.strip()
    return text

# 讀取推文資料
data1 = []
with open('/kaggle/input/dm-2024-isa-5810-lab-2-homework/tweets_DM.json', 'r') as f:
    for line in f:
        data1.append(json.loads(line))

# 處理每條推文並儲存結果
processed_tweets = []
for entry in data1:
    # 檢查 '_source' 和 'tweet' 是否存在於記錄中
    if '_source' in entry and 'tweet' in entry['_source']:
        tweet = entry['_source']['tweet']
        # 檢查 'text' 是否存在於 'tweet' 中
        if 'text' in tweet:
            tweet_text = tweet['text']
            cleaned_text = clean_tweet(tweet_text, emoji_dict)
            # 創建處理後的推文記錄，保留 '_source' 和 'tweet'
            processed_tweet = {
                '_source': {
                    'tweet': tweet.copy()
                }
            }
            # 更新清理後的文本
            processed_tweet['_source']['tweet']['text'] = cleaned_text
            processed_tweets.append(processed_tweet)
        else:
            print("記錄中缺少 'text' 鍵")
    else:
        print("記錄中缺少 '_source' 或 'tweet' 鍵")

# 將處理後的推文資料存儲為 JSON 檔案
with open('/kaggle/working/tweets_DM_filtered_1.json', 'w') as outfile:
    json.dump(processed_tweets, outfile, ensure_ascii=False, indent=4)



# 將處理後的資料轉換為 DataFrame
df_processed = pd.DataFrame(processed_tweets)

# 定義輸出目錄和檔案名稱
output_dir = '/kaggle/working/'
output_file = 'tweets_DM_filtered_1.json'

# 檢查並創建目錄（如果不存在）
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 將處理後的 DataFrame 儲存為 JSON 檔案
output_path = os.path.join(output_dir, output_file)
df_processed.to_json(output_path, orient='records', lines=True, force_ascii=False)

# 把 /kaggle/working/tweets_DM_filtered_1.json 載入成為 data , 可樣就可以跟原本程式合併

data = []
with open('/kaggle/working/tweets_DM_filtered_1.json', 'r') as f:
    for line in f:
        data.append(json.loads(line))
f.close()

# 以下就都跟之前一樣了

In [ ]:
# 2. 讀取其他 CSV 文件
emotion = pd.read_csv('/kaggle/input/dm-2024-isa-5810-lab-2-homework/emotion.csv')
data_identification = pd.read_csv('/kaggle/input/dm-2024-isa-5810-lab-2-homework/data_identification.csv')

# ======== 數據處理 ========
# 1. 從 JSON 數據提取 tweet_id, hashtags, 和文本內容
df = pd.DataFrame(data)
_source = df['_source'].apply(lambda x: x['tweet'])
df = pd.DataFrame({
    'tweet_id': _source.apply(lambda x: x['tweet_id']),
    'hashtags': _source.apply(lambda x: x['hashtags']),
    'text': _source.apply(lambda x: x['text']),
})

In [ ]:


# 2. 合併資料集標籤 (train/test)
df = df.merge(data_identification, on='tweet_id', how='left')

# 3. 將資料分為訓練集和測試集
train_data = df[df['identification'] == 'train']
test_data = df[df['identification'] == 'test']

# 4. 合併情緒標籤到訓練資料中
train_data = train_data.merge(emotion, on='tweet_id', how='left')

# 5. 移除訓練集中重複的文本
train_data.drop_duplicates(subset=['text'], keep=False, inplace=True)

# ======== 訓練數據準備 ========
# 1. 抽取部分訓練數據（30% 抽樣）
train_data_sample = train_data.sample(frac=0.3, random_state=42)

# 2. 提取目標變數 (情緒) 和特徵變數 (文本)
y_train_data = train_data_sample['emotion']
X_train_data = train_data_sample.drop(['tweet_id', 'emotion', 'identification', 'hashtags'], axis=1)

# 3. 訓練集與測試集劃分 (分層抽樣確保類別分佈一致)
X_train, X_test, y_train, y_test = train_test_split(
    X_train_data, y_train_data, test_size=0.2, random_state=42, stratify=y_train_data
)

# ======== 文本特徵轉換 ========
# 1. 使用 TF-IDF 將文本轉換為數值特徵
tfidf = TfidfVectorizer(max_features=500)
X = tfidf.fit_transform(X_train['text']).toarray()
X_test = tfidf.transform(X_test['text'])

# ======== 標籤編碼 ========
# 1. 將情緒類別轉換為數值型
le = LabelEncoder()
y = le.fit_transform(y_train)
y_test = le.transform(y_test)

# ======== 模型訓練 ========
# 1. 訓練隨機森林分類器
clf = RandomForestClassifier()
clf.fit(X, y)

# ======== 模型評估 ========
# 1. 預測測試集
y_pred = clf.predict(X_test)

# 2. 計算準確率
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")

# ======== 測試數據預測 ========
# 1. 準備測試數據的文本特徵
X_test_data = test_data.drop(['tweet_id', 'identification', 'hashtags'], axis=1)
X_test_data = tfidf.transform(X_test_data['text']).toarray()

# 2. 預測測試數據情緒
y_test_pred = clf.predict(X_test_data)

# ======== 結果輸出 ========
# 1. 將數值標籤轉換回文字標籤
y_pred_labels = le.inverse_transform(y_test_pred)

# 2. 保存提交檔案
submission = pd.DataFrame({
    'tweet_id': test_data['tweet_id'],
    'emotion': y_pred_labels
})

# 先不產生 submission
# submission.to_csv('/kaggle/working/submission.csv', index=False)


In [ ]:
submission.head()

# 分析部分 : 混淆矩陣 , Classification Report

# 1. TF-IDF + Randon_forest
## 準確率紀錄 :
### 未做雜訊處理前
- Accuracy: 0.4799
- Accuracy: 0.4808

### 做過雜訊處理後
- Accuracy: 0.4717
- Accuracy: 0.4708

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# ======== 分析部分 ========

# 1. 列出混淆矩陣
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)
print("\n", "----", "\n")

# 2. 畫出混淆矩陣
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=le.classes_)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()
print("\n", "----", "\n")

# 3 . Classification Report
class_report = classification_report(y_test, y_pred, target_names=le.classes_)
print("Classification Report:")
print(class_report)
print("\n", "----", "\n")

# 4. 印出 Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print("\n", "----", "\n")